This notebook works through the process of pulling documents references in from Zotero collections that we are using as part of our GeoArchive efforts. These documents provide reference materials on mineral exploration history and other details important to our use cases. The GeoKB serves as a point where all of the pertinent assertions derived from these documents come together and link to other information for research and analysis.

We use Zotero because it provides a platform for effectivey managing the documents and their metadata, giving scientists and science support personnel a reasonable set of reference management tools. It also provides a solid API to operate against when we need to pull reference details into other systems.

### Pre-cached Inventory

The Zotero Python API (pyzotero) provides a number of methods for searching and working with libraries, collections, and items. We use Group Libraries as logical containers that are considered "GeoArchive Collections." Zotero libraries can be further organized into collections (folders) that can be hierarchical as a convenient management mechanism.

Once a library gets big enough with potentially thousands of items, it can be somewhat time consuming to pull everything into some other platform like the GeoKB (.everything() method in pyzotero handles this politely but slowly). For this reason, we built a utility function that periodically refreshes a cached inventory with all metadata for the library, storing this as a file within a specialty item in the library. When we want to grab everything for some purpose, we can simply get the inventory and work with it from there.

### Entity Identification and Extraction

One of the value-added services we layer on top of GeoArchive colections is a process to pipe contents through one or more engines for identifying important entities from within the archived materials and extracting them for further evaluation and use in research and assessments. Currently, we pipe all Zotero-based collections through the xDD (GeoDeepDive) engine at the University of Wisconsin-Madison. xDD processing includes a combination of search indexing and natural language processing (NLP) methods along with identification/extraction of tables, figures, and equations.

We're still developing the methodology to leverage the xDD processing pipelines to best effect. This will include pulling out key concepts from specific "dictionaries" and feeding those back to Zotero as tags as well as pulling document-specific extractions and dropping those as additional file attachments. These actions will roundtrip extracted information into the platform where the documents are being managed so that they are readily available in context for use.

The GeoKB piece of the architecture provides the platform where these extractions can be dynamically curated within a larger context of every other piece of related/linked information. We still need to work out where all the pieces of technology and processing software need to sit in order to handle this reliably and efficiently, but this notebook provides a start to how it all makes its way into the GeoKB.

In [1]:
import os
from pyzotero import zotero
import pandas as pd

In [2]:
ni43101_library_id = '4530692'
ni43101_api_key = os.environ['NI43101_KEY']

### Connecting to Zotero

An API key along with a group library identifier are necessary parameters. Our seiminal use case is a collection of [National Instrument (NI) 43-101 Technical Reports](https://www.zotero.org/groups/4530692/usgs_ni_43-101_reports/library) that has been developed over many years and was recently brought into a Zotero group library for continued development, management, and use.

In [3]:
ni43101_library = zotero.Zotero(
    ni43101_library_id,
    'group', 
    ni43101_api_key
)

### Collections

Larger libraries in Zotero often benefit from being organized into collections (or folders). Collections aid in management operations by creating logical groupings to operate within, and they can help in navigating through materials. From a knowledgebase perspective, collections may provide a useful piece of information about the documents within the collections that we need to parse and bring into the knowledgebase context.

For our seminal use case with the NI 43-101 Technical Reports, we are organizing them into collections based on the geographic region for the mine or prospect documented within the report.

In [4]:
ni43101_collections = ni43101_library.all_collections()

In [5]:
df_ni43101_collections = pd.DataFrame([i['data'] for i in ni43101_collections])
df_ni43101_collections.head(8)

,key,version,name,parentCollection,relations
0,U4M9GAQX,45944,South America,False,{}
1,EFK6U3JQ,45951,Guyana,U4M9GAQX,{}
2,QN7CTH5X,45950,Ecuador,U4M9GAQX,{}
3,VFBJ2RZN,45949,Peru,U4M9GAQX,{}
4,92RBMDVE,45948,Chile,U4M9GAQX,{}
5,H4CWF54J,45947,Argentina,U4M9GAQX,{}
6,A2ZQSJGF,45946,Brazil,U4M9GAQX,{}
7,82HRAV4I,45945,Colombia,U4M9GAQX,{}


#### Notes

We will want to think through whether to incorporate these place names into the GeoKB. Ultimately, we want to have one or more point coordinate locations associated with a given document representing the mining property(ies) documented in a report. If we can get all the way down to that point through data extraction processing, we could conceivably build up additional place name identification including these same high level orientation tags used for document organization. Since we don't have that right now, or at least not comprehensively, we might at least start with these to model out how the information goes into the GeoKB source. We'll need to add in the higher level grouping at continental scale (e.g., South America) to work we started in the Place Name Reference notebook on country identification.

### Item Types and Information Models

Zotero uses a couple of different standards developed in the library community over many years to inform its model of item types and property schemas. Our goal in bringing these references into the GeoKB is not to duplicate everything from Zotero. We need to have something recognizable as a label and description for document items along with a clear pointer into the Zotero web interface to get further information, but we are mostly focused on the information derived from the documents pertinent to our mineral assessment use cases and the linkages to other things in the knowledgebase (e.g., identification of mineral occurrences and their documented characteristics).

We do need to go through the schemas we will be employing from Zotero and work out applicable mapping to GeoKB properties, building out additional reference sources as needed.

In [6]:
zotero_item_types = ni43101_library.item_types()
[i['localized'] for i in zotero_item_types]

['Artwork',
 'Audio Recording',
 'Bill',
 'Blog Post',
 'Book',
 'Book Section',
 'Case',
 'Conference Paper',
 'Dictionary Entry',
 'Document',
 'E-mail',
 'Encyclopedia Article',
 'Film',
 'Forum Post',
 'Hearing',
 'Instant Message',
 'Interview',
 'Journal Article',
 'Letter',
 'Magazine Article',
 'Manuscript',
 'Map',
 'Newspaper Article',
 'Note',
 'Patent',
 'Podcast',
 'Preprint',
 'Presentation',
 'Radio Broadcast',
 'Report',
 'Software',
 'Statute',
 'TV Broadcast',
 'Thesis',
 'Video Recording',
 'Web Page']

#### All Fields

In [7]:
zotero_item_properties = ni43101_library.item_fields()
for x in [i['localized'] for i in zotero_item_properties]:
    print(x)

# of Pages
# of Volumes
Abstract
Accessed
Application Number
Archive
Archive ID
Artwork Size
Assignee
Bill Number
Blog Title
Book Title
Call Number
Case Name
Citation Key
Code
Code Number
Code Pages
Code Volume
Committee
Company
Conference Name
Country
Court
DOI
Date
Date Decided
Date Enacted
Dictionary Title
Distributor
Docket Number
Document Number
Edition
Encyclopedia Title
Episode Number
Extra
File Type
Filing Date
First Page
Format
Format
Forum/Listserv Title
Genre
History
ISBN
ISSN
Institution
Issue
Issue Date
Issuing Authority
Journal Abbr
Label
Language
Legal Status
Legislative Body
Library Catalog
Loc. in Archive
Medium
Medium
Meeting Name
Name of Act
Network
Pages
Patent Number
Place
Post Type
Priority Numbers
Proceedings Title
Prog. Language
Program Title
Public Law Number
Publication
Publisher
References
Report Number
Report Type
Reporter
Reporter Volume
Repository
Rights
Running Time
Scale
Section
Series
Series Number
Series Text
Series Title
Session
Short Title
Studio
Sub

#### Report Fields

In [8]:
zotero_report_properties = ni43101_library.item_type_fields('report')
for x in [i['localized'] for i in zotero_report_properties]:
    print(x)

Title
Abstract
Report Number
Report Type
Series Title
Place
Institution
Date
Pages
Language
Short Title
URL
Accessed
Archive
Loc. in Archive
Library Catalog
Call Number
Rights
Extra


#### Notes

* Institution - For NI 43-101 Reports, we may establish a convention that this field applies to the commercial company owning the mining property described in the report. This would align with the exploration we are conducting on the similar SEC S-K 1300 reports. If this links by name to a uniquely named "commercial company" entity in the GeoKB, we can exploit additional information there (CIK identifiers, LEI identifiers, etc.) to go after further information sources such as other legal filings, news releases, or others that we can exploit for further intelligence gathering.
* Source identifiers/access points - Wherever possible, we should follow a "shortest path" principle in bringing document-type items into the GeoKB such that interacting with the knowledge graph includes being able to get to source material without unnecessary "clicks." For things like the NI 43-101 reports where we are using online Zotero storage for the original documents, we should incorporate those links into the GeoKB items (even if they may require authentication to access).
* Place - We will need to put some thought into how best to handle place names in our model. We are currently organizing documents into place name collections at the level of continental region, country name, and U.S. State name. Tags could provide another useful place to store place names (probably more functional than collections). Since they do help in organizing and orienteering through a library, storing at least some types of place names as metadata makes sense within Zotero, but we could also store detailed geographic area information within attached files (e.g., GeoJSON). In any case, we need to work out the logical mapping from however we do this to the GeoKB representation of these documents.
* Authors (part of the common schema in Zotero) - Authors in Zotero are identified by either full name or name parts in typical citation fashion. Better identification available in some citation metadata schemas like ORCID are not currently part of the Zotero schemas. We will have a representation for many people in the GeoKB, at least in terms of USGS publishing authors, dataset contributors, etc. We'll have to determine whether it is useful to establish explicit linkages to author items in the GeoKB from document items sourced from the GeoArchive. This is somewhat moot for NI 43-101 reports unless and until we get better author information into the system.

### Tags

Tags in a Zotero library provide a ready means to identify key aspects of the library contents with the ability to filter and browse. We are experimenting with a piece of software to pull useful tags from the xDD pipelines into the source Zotero library to help make that extracted information readily usable in their primary document management and access context. But tags can also be added by individual authorized users with write access to the Zotero library.

Some or all of these tags will be useful as linked entities within the GeoKB. We could tune the same "middleware" operating against the xDD API to feed information into both Zotero libraries and the GeoKB, but because of the multi-modal (human and robot) dynamic of tagging in Zotero, we may want to use Zotero as the initial foundation and then move tags from there into linked items in the GeoKB. One way or the other, we need to develop the conventions and rules on how the tags are built and processed to give us all the functionality we need.

In [10]:
ni43101_tags = ni43101_library.tags()
ni43101_tags[:20]

['xDD minerals:Chapmanite',
 'xDD minerals:Cobaltpentlandite',
 'xDD stratigraphic_names:Angostura Formation',
 'xDD minerals:Pyrochroite',
 'xDD minerals:Uchucchacuaite',
 'xDD minerals:Lampadite',
 'xDD minerals:Stannoidite',
 'xDD minerals:Quenselite',
 'xDD minerals:Spangolite',
 'xDD stratigraphic_names:Shoal Arm Formation',
 'xDD stratigraphic_names:Gleeson Quartz Monzonite',
 'xDD stratigraphic_names:Turquoise Granite',
 'xDD stratigraphic_names:Stronghold Granite',
 'xDD minerals:Carminite',
 'xDD stratigraphic_names:Hazelwood Formation',
 'xDD stratigraphic_names:Wheeler Formation',
 'xDD stratigraphic_names:Wheeler Shale',
 'xDD stratigraphic_names:Zabriskie Formation',
 'xDD stratigraphic_names:Banded Mountain Member',
 'xDD stratigraphic_names:Dunderberg Shale Member']

In [11]:
tag_prefixes = list(set([i.split(':')[0] for i in ni43101_tags if ':' in i]))
tag_prefixes

['xDD stratigraphic_names', 'xDD minerals']

### Attachments

Zotero items can have any number of file attachments of any kind. For the NI 43-101 library, primary files are the raw PDF report content pulled from the Canadian Government portal or some other site. As we work through pulling the most useful extractions together from the xDD pipelines or other processing mechanisms, we may store additional files, including structured data files such as JSON or GeoJSON.

We are also working on protocols for individual document annotation and highlighting. Zotero libraries/items can be processed using the Zotfile extension to extract annotation and highlights to Markdown files that are attached to the items. These attachments can then be processed with code to introduce linked entities or other types of claims to the GeoKB.

In [12]:
top_items = ni43101_library.top()
top_items

[{'key': 'ETIGHE89',
  'version': 67378,
  'library': {'type': 'group',
   'id': 4530692,
   'name': 'USGS NI 43-101 Reports',
   'links': {'alternate': {'href': 'https://www.zotero.org/groups/usgs_ni_43-101_reports',
     'type': 'text/html'}}},
  'links': {'self': {'href': 'https://api.zotero.org/groups/4530692/items/ETIGHE89',
    'type': 'application/json'},
   'alternate': {'href': 'https://www.zotero.org/groups/usgs_ni_43-101_reports/items/ETIGHE89',
    'type': 'text/html'},
   'enclosure': {'type': 'application/pdf',
    'href': 'https://api.zotero.org/groups/4530692/items/ETIGHE89/file/view',
    'title': 'Nassar et al. - 2022 - Rock-to-Metal Ratio A Foundational Metric for Und.pdf',
    'length': 29987278}},
  'meta': {'createdByUser': {'id': 1119084,
    'username': 'skybristol',
    'name': 'Sky Bristol',
    'links': {'alternate': {'href': 'https://www.zotero.org/skybristol',
      'type': 'text/html'}}},
   'numChildren': 0},
  'data': {'key': 'ETIGHE89',
   'version': 67

In [15]:
top_items[0]['links']

{'self': {'href': 'https://api.zotero.org/groups/4530692/items/ETIGHE89',
  'type': 'application/json'},
 'alternate': {'href': 'https://www.zotero.org/groups/usgs_ni_43-101_reports/items/ETIGHE89',
  'type': 'text/html'},
 'enclosure': {'type': 'application/pdf',
  'href': 'https://api.zotero.org/groups/4530692/items/ETIGHE89/file/view',
  'title': 'Nassar et al. - 2022 - Rock-to-Metal Ratio A Foundational Metric for Und.pdf',
  'length': 29987278}}

#### Notes

* Direct document links - As discussed above, we may want to identify and pull in the primary link to a document into the GeoKB representation for these reports so that applications built on the GeoKB can link right to the source in addition to going to/through the library metadata item.